# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Подготовка данных

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv('Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Видно, что данные уже корректно приведены в нужные типы данных, единственное (нет), что надо решить - это проблему с пропусками в столбце Tenure. Заполним данную графу средними значениями. А также преобразуем методом OHE категориальные признаки по географии. Столбец Surname нам также не нужен, ROWNumber И CustomerID нам не нужны для прогнозирования

In [4]:
data['Tenure'].fillna(data['Tenure'].mean(), inplace=True)
data.drop('Surname', axis=1, inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  float64
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 1015.8+ KB


In [6]:
data_ohe = pd.get_dummies(data) # Преобразовываем категориальные признаки
data_ohe.drop(['RowNumber', 'CustomerId'], axis=1, inplace=True)

In [7]:
data_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1,1,0


Данные готовы

# 2. Исследование задачи

Спрогнозируем нужные значения на сырых данных, параметры у RandomForest уже подобраны. Для этого поделим выбору на три части: Обучающуюся, валидационную и тестовую

In [8]:
target = data_ohe['Exited']
features = data_ohe.drop('Exited', axis=1)
features_train, features_vt, target_train, target_vt = train_test_split(features, target, 
                                                                        test_size=0.4, random_state = 82357)
features_valid, features_test, target_valid, target_test = train_test_split(features_vt, target_vt, 
                                                                        test_size=0.5, random_state = 8357)

In [9]:
%%time
best_f1 = 0
best_model = 0
best_est_and_depth = [0,0]
#for est in range(60, 80):
    #for dp in range(7, 15):
model = RandomForestClassifier(n_estimators = 66, max_depth = 13,
                               random_state=35)
model.fit(features_train, target_train)
predict = model.predict(features_valid)
f1 = f1_score(target_valid,predict)
if f1 > best_f1:
    best_f1 = f1
    best_model = model
    best_est_and_depth = [66, 13]
print("F1 non_normilized data =", best_f1)

F1 non_normilized data = 0.5448717948717948
Wall time: 950 ms


In [10]:
probabilities = best_model.predict_proba(features_valid)
auc_roc = roc_auc_score(target_valid, probabilities[:,1])
print('Auc_roc of model', auc_roc)

Auc_roc of model 0.8386734492685043


Без учета дисбаланся f1 ниже установленного заказчиком порога, поэтому надо бы все это учесть и доработать (ниже)

# 3. Борьба с дисбалансом

In [26]:
target_train.value_counts(normalize=True) 
# Нулей в 4 раза больше 1. То есть виден ярко-выраженный дисбаланс классов. Исправим

0    0.794833
1    0.205167
Name: Exited, dtype: float64

In [12]:
data_ohe_2 = data_ohe.copy()
data_ohe_2 # на всякий случай скопируем датасет

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2.00000,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1.00000,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8.00000,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1.00000,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2.00000,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.00000,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10.00000,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7.00000,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3.00000,75075.31,2,1,0,92888.52,1,0,1,0,0,1


In [13]:
numbers = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary'] # нормализоывваем численные признаки
scl = StandardScaler()
data_ohe_2[numbers] = scl.fit_transform(data_ohe_2[numbers])
data_ohe_2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,-0.326221,0.293517,-1.086170,-1.225848,-0.911583,1,1,0.021886,1,1,0,0,1,0
1,-0.440036,0.198164,-1.448505,0.117350,-0.911583,0,1,0.216534,0,0,0,1,1,0
2,-1.536794,0.293517,1.087844,1.333053,2.527057,1,0,0.240687,1,1,0,0,1,0
3,0.501521,0.007457,-1.448505,-1.225848,0.807737,0,0,-0.108918,0,1,0,0,1,0
4,2.063884,0.388871,-1.086170,0.785728,-0.911583,1,1,-0.365276,0,0,0,1,1,0


In [14]:
# теперь поделим данные на три выборке, но уже при нормализованных данных
features2 = data_ohe_2.drop('Exited', axis=1)
features_train2, features_vt2, target_train2, target_vt2 = train_test_split(features2, target, 
                                                                        test_size=0.4, random_state = 82357)
features_valid2, features_test2, target_valid2, target_test2 = train_test_split(features_vt2, target_vt2, 
                                                                        test_size=0.5, random_state = 8357)

In [15]:
# приводим классы обучающейся выборки в равные доли методом upsample
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_train_up, target_train_up=upsample(features_train2, target_train2, 4)
print(features_train_up.shape)
print(target_train_up.shape)

(9693, 13)
(9693,)


In [16]:
%%time
## Модель upsample

best_f1 = 0
best_model = 0
best_est_and_depth = [37,12]
#for est in range(10, 40):
    #for dp in range(12, 18):
model = RandomForestClassifier(n_estimators = 37, max_depth = 12,
                               random_state=24504)
model.fit(features_train_up, target_train_up)
predict = model.predict(features_valid2)
f1 = f1_score(target_valid2,predict)
if f1 > best_f1:
    best_f1 = f1
    best_model = model
    best_est_and_depth = [37, 12]
print('f1_upsample:', best_f1)
print(best_est_and_depth)
probabilities_2 = best_model.predict_proba(features_valid2)
auc_roc_2 = roc_auc_score(target_valid2, probabilities_2[:,1])

print('Auc_roc umsample equals to', auc_roc)

f1_upsample: 0.6032540675844806
[37, 12]
Auc_roc umsample equals to 0.8386734492685043
Wall time: 776 ms


Получена довольно-таки неплохая f1, видно, что при восстановлении баланса качество модели резко возрастает

In [17]:
%%time
## Модель class_weight
best_f1_3 = 0
best_model_3 = 0
best_est_and_depth_3 = [0,0]
#for est in range(20, 85):
   # for dp in range(8, 15):
model = RandomForestClassifier(n_estimators = 36, max_depth = 9,
                               random_state=2404, class_weight='balanced')
model.fit(features_train2, target_train2)
predict_cw = model.predict(features_valid2)
f1 = f1_score(target_valid2,predict_cw)
if f1 > best_f1_3:
    best_f1_3 = f1
    best_model_3 = model
    best_est_and_depth_3 = [36, 9]
print('f1_class_wieght_method:', best_f1_3)
print(best_est_and_depth_3)
probabilities_3 = best_model_3.predict_proba(features_valid2)
auc_roc_2 = roc_auc_score(target_valid2, probabilities_3[:,1])

print('Auc_roc_cw_method_equals_to', auc_roc)

f1_class_wieght_method: 0.5978391356542617
[36, 9]
Auc_roc_cw_method_equals_to 0.8386734492685043
Wall time: 474 ms


Также работает неплохо, чуть уступает на валидационной выборке модели выше.

In [18]:
## Model downSample
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

fetures_downsample, target_downsample = downsample(features_train2, target_train2, 0.25)
print(fetures_downsample.shape)
print(target_downsample.shape)

(2423, 13)
(2423,)


In [19]:

best_f1_d = 0
best_model_d = 0
best_par_d = [30, 8]
#for i in range(20, 88):
    #for j in range(7, 15):
md = RandomForestClassifier(n_estimators = 30, max_depth = 8,
                               random_state=2301234)
md.fit(fetures_downsample,target_downsample)
predict_d = md.predict(features_valid2)
f1 = f1_score(target_valid2,predict_d)
if f1 > best_f1_d:
    best_f1_d = f1
    best_model_d = md
    #best_par_d = [i, j]
print('f1_downsample =',best_f1_d)
print(best_par_d)
print("AUC_ROC_sownsample",roc_auc_score(target_test2, best_model_d.predict_proba(features_test2)[:,1]))

f1_downsample = 0.594
[30, 8]
AUC_ROC_sownsample 0.8384728340675479


Самая худшая из трех моделей по f1, но не сильно остает

# 4. Тестирование модели

In [24]:
# Тестируем модель upsample на f1 и auc
f_pr=best_model.predict(features_test2)
f_f1=f1_score(target_test2,f_pr)
probabilities_f = best_model.predict_proba(features_test2)
a3 = roc_auc_score(target_test2, probabilities_f[:,1])
print(f'final_f1_umsample_model:',f_f1, f'\nAuc_Roc_upsample= {a3}') #With upsample

final_f1_umsample_model: 0.5875706214689265 
Auc_Roc_upsample= 0.8351133630081506


Как видно, модель переобучилась и не прошла порог

In [21]:
print("Лучшая модель, которая дает нужный результат: \n")
print('Final f1 class_weight_model:',f1_score(target_test2, best_model_3.predict(features_test2)))
print('Auc_roc_class_weight = ', roc_auc_score(target_test2, best_model_3.predict_proba(features_test2)[:,1]))

Лучшая модель, которая дает нужный результат: 

Final f1 class_weight_model: 0.5949506037321625
Auc_roc_class_weight =  0.8416914339612818


In [22]:
print('Downsample model f1:',f1_score(target_test2, best_model_d.predict(features_test2)))
print('Auc_of_downsample_model=', roc_auc_score(target_test2, best_model_d.predict_proba(features_test2)[:,1]))

Downsample model f1: 0.5890151515151514
Auc_of_downsample_model= 0.8384728340675479


# Итого
Модель best_model_3, сбалансированная просто-напросто указанием соответствующего параметра о дисбалансе, дала лучший результат. мы получили F1=0.595. Auc_roc = 0.84 на тестовой выборке

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*